In [1]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 8.5 MB/s eta 0:00:00


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.4 MB/s eta 0:00:00


In [4]:
!pip install flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split, GridSearchCV, KFold

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingRegressor
from catboost import CatBoostRegressor
import xgboost as xgb

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error

from scipy.stats import norm
import numpy as np
import tensorflow as tf

In [6]:
url = "https://raw.githubusercontent.com/Tejanikhil/Shared-ASAG/main"
df = pd.read_csv(url + "/SemEvalData.csv")

In [7]:
import pandas as pd
import nltk
import numpy as np
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords

# Download the stopwords corpus if not already downloaded
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [8]:
def extract_pos(text):
    tokens = word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    pos_words = [word for word, pos in pos_tags if pos.startswith('VB') or pos.startswith('JJ') or pos.startswith('NN')]
    return ' '.join(pos_words)

def process_dataset(text):
    lower_cased = text.lower()
    tokens = nltk.word_tokenize(lower_cased)
    filtered_tokens = [token for token in tokens if token not in stop_words]
    preprocessed = ' '.join(filtered_tokens)
    return preprocessed

In [9]:
df["concatenated"] = df['Question'] + " " + df['Student Answer']

In [10]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(df.loc[:,["concatenated"]], df["Grade"], test_size=0.1, random_state=42)

In [11]:
TRAIN = train_X
TRAIN["LABELS"] = train_y
TEST = test_X
TEST["LABELS"] = test_y

In [12]:
train_X["concatenated"] = train_X["concatenated"].apply(extract_pos).apply(process_dataset)

test_X["concatenated"] = test_X["concatenated"].apply(extract_pos).apply(process_dataset)

In [13]:
print(TRAIN.shape)
TRAIN = TRAIN.replace("", pd.NA).dropna()
print(TRAIN.shape)
TEST = TEST.replace("", pd.NA).dropna()

(2197, 2)
(2197, 2)


In [14]:
train_X = TRAIN.drop("LABELS", axis = 1)
train_y = TRAIN.iloc[:,-1]

In [15]:
sentences = list(train_X["concatenated"])

In [16]:
rm -rf ~/.cache/huggingface/transformers/bert-base-uncased

In [17]:
from flair.embeddings import FlairEmbeddings
from flair.data import Sentence
import flair

In [18]:
flair_forward = flair.embeddings.FlairEmbeddings('multi-forward')
flair_backward = flair.embeddings.FlairEmbeddings('multi-backward')
flair_en_forward = flair.embeddings.FlairEmbeddings('en-forward')

2023-05-28 14:36:52,578 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/lm-jw300-forward-v0.1.pt not found in cache, downloading to /tmp/tmp46ye2kqh


100%|██████████| 165M/165M [00:04<00:00, 39.0MB/s]

2023-05-28 14:36:57,087 copying /tmp/tmp46ye2kqh to cache at /root/.flair/embeddings/lm-jw300-forward-v0.1.pt


2023-05-28 14:36:57,291 removing temp file /tmp/tmp46ye2kqh
2023-05-28 14:37:06,104 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/lm-jw300-backward-v0.1.pt not found in cache, downloading to /tmp/tmp1i3_wwmm


100%|██████████| 165M/165M [00:04<00:00, 38.3MB/s]

2023-05-28 14:37:10,696 copying /tmp/tmp1i3_wwmm to cache at /root/.flair/embeddings/lm-jw300-backward-v0.1.pt


2023-05-28 14:37:10,895 removing temp file /tmp/tmp1i3_wwmm
2023-05-28 14:37:11,313 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/news-forward-0.4.1.pt not found in cache, downloading to /tmp/tmp1w6fzpbe


100%|██████████| 69.7M/69.7M [00:02<00:00, 35.9MB/s]

2023-05-28 14:37:13,432 copying /tmp/tmp1w6fzpbe to cache at /root/.flair/embeddings/news-forward-0.4.1.pt


2023-05-28 14:37:13,507 removing temp file /tmp/tmp1w6fzpbe


In [19]:
flair_news_forward = flair.embeddings.FlairEmbeddings('news-forward')
flair_news_backward = flair.embeddings.FlairEmbeddings('news-backward')

2023-05-28 14:37:14,039 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/news-backward-0.4.1.pt not found in cache, downloading to /tmp/tmpf57hiirv


100%|██████████| 69.7M/69.7M [00:02<00:00, 36.1MB/s]

2023-05-28 14:37:16,138 copying /tmp/tmpf57hiirv to cache at /root/.flair/embeddings/news-backward-0.4.1.pt


2023-05-28 14:37:16,228 removing temp file /tmp/tmpf57hiirv


In [20]:
train_Embeddings_df = pd.DataFrame({"Q_Student_Answer_Embeddings":train_X["concatenated"]})
test_Embeddings_df = pd.DataFrame({"Q_Student_Answer_Embeddings":test_X["concatenated"]})

In [21]:
def get_flair_multi_forward_embeddings(text):
  sentence = Sentence(text)
  flair_forward.embed(sentence)
  embeds = [np.array(i.embedding.cpu()) for i in sentence]
  return np.mean(np.array(embeds),axis = 0)

In [22]:
def get_flair_multi_backward_embeddings(text):
  sentence = Sentence(text)
  flair_backward.embed(sentence)
  embeds = [np.array(i.embedding.cpu()) for i in sentence]
  return np.mean(np.array(embeds),axis = 0)

In [23]:
def get_flair_news_forward_embeddings(text):
  sentence = Sentence(text)
  flair_news_forward.embed(sentence)
  embeds = [np.array(i.embedding.cpu()) for i in sentence]
  return np.mean(np.array(embeds),axis = 0)

In [24]:
def get_flair_news_backward_embeddings(text):
  sentence = Sentence(text)
  flair_news_backward.embed(sentence)
  embeds = [np.array(i.embedding.cpu()) for i in sentence]
  return np.mean(np.array(embeds),axis = 0)

In [25]:
def get_flair_en_forward_embeddings(text):
  sentence = Sentence(text)
  flair_en_forward.embed(sentence)
  embeds = [np.array(i.embedding.cpu()) for i in sentence]
  return np.mean(np.array(embeds),axis = 0)

In [26]:
train_Embeddings_df["Q_Student_Answer_Embeddings"] = train_Embeddings_df["Q_Student_Answer_Embeddings"].apply(get_flair_en_forward_embeddings)

test_Embeddings_df["Q_Student_Answer_Embeddings"] = test_Embeddings_df["Q_Student_Answer_Embeddings"].apply(get_flair_en_forward_embeddings)


In [27]:
train_Embeddings_df

,Q_Student_Answer_Embeddings
199,"[0.0020029247, 0.0001697924, 0.033528276, -0.0..."
229,"[0.0005199325, -0.00018717712, 0.036420085, 0...."
521,"[0.00026043254, -6.446933e-05, 0.053852882, 0...."
651,"[-0.00031311344, 0.00033376226, 0.02394788, 0...."
450,"[0.0006915508, 0.00037120495, 0.08009664, -0.0..."
...,...
1638,"[-9.836216e-05, 0.003379664, 0.041305516, 0.00..."
1095,"[0.0034226782, -0.00033975905, 0.026702158, 0...."
1130,"[-0.0034282154, 0.0058426154, 0.08534176, 0.02..."
1294,"[0.004336436, 0.003268753, 0.06238764, 0.00851..."


In [28]:
train_X["QSA_Embeddings"] = train_Embeddings_df.apply(lambda x:x["Q_Student_Answer_Embeddings"], axis = 1)
test_X["QSA_Embeddings"] = test_Embeddings_df.apply(lambda x:x["Q_Student_Answer_Embeddings"], axis = 1)

In [29]:
train_df = pd.DataFrame(list(train_X["QSA_Embeddings"]))
test_df = pd.DataFrame(list(test_X["QSA_Embeddings"]))

In [30]:
train_df.to_csv("flair_train_embeddings_QSA.csv")
test_df.to_csv("flair_test_embeddings_QSA.csv")

In [31]:
pca = PCA(n_components=100) 
pca.fit_transform(train_df)
principal_components = pca.transform(train_df)
train_df_pca = pd.DataFrame(data=principal_components)

In [32]:
from sklearn.linear_model import LinearRegression
LR_model = LinearRegression()
LR_model.fit(train_df, train_y)

LinearRegression()

In [33]:
preds = LR_model.predict(test_df)
y_preds = [round(y_val, 1) for y_val in preds]
actuals = test_y
y_true = [round(y_val, 1) for y_val in actuals]

In [34]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
LR_mae = mean_absolute_error(y_true, np.round(y_preds, 1))
LR_mse = mean_squared_error(y_true, np.round(y_preds, 1))
LR_rmse = mean_squared_error(y_true, np.round(y_preds, 1), squared=False)
LR_mape = mean_absolute_percentage_error(y_true, y_preds, multioutput = "uniform_average")
LR_r2 = r2_score(y_true, y_preds)

In [35]:
print("Mean Absolute Error (MAE): ", np.round(LR_mae, 2))
print("Mean Squared Error (MSE): ", np.round(LR_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(LR_rmse, 2))
print("Mean Absolute Percentage Error (MAPE): ", np.round(LR_mape, 2))
print("R2 Score: ", np.round(LR_r2, 2))

Mean Absolute Error (MAE):  17.99
Mean Squared Error (MSE):  675.03
Root Mean Squared Error (RMSE):  25.98
Mean Absolute Percentage Error (MAPE):  3121270263811081.0
R2 Score:  -335.35


In [36]:
from sklearn import neighbors
rmse_val = []
mae_val = []
mse_val = []
r2_val = []
for K in range(2,20):
    knn_model = neighbors.KNeighborsRegressor(n_neighbors = K)
    knn_model.fit(train_df, train_y)  #fit the model
    pred=knn_model.predict(test_df) #make prediction on test set
    
    mae = mean_absolute_error(y_true, np.round(pred, 1))
    mae_val.append(mae)
    
    mse = mean_squared_error(y_true, np.round(pred, 1))
    mse_val.append(mse)
    
    rmse = mean_squared_error(y_true, np.round(pred, 1), squared=False)
    rmse_val.append(rmse)
    
    r2 = r2_score(y_true, pred)
    r2_val.append(r2)

In [37]:
y_true = test_y
knn_preds = knn_model.predict(test_df)

In [38]:
min_val = min(r2_val)
index = r2_val.index(min_val)
knn_r2 = min_val
knn_rmse = rmse_val[index]
knn_mae = mae_val[index]
knn_mse = mse_val[index]

In [39]:
print("Mean Absolute Error (MAE): ", np.round(knn_mae, 2))
print("Mean Squared Error (MSE): ", np.round(knn_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(knn_rmse, 2))
print("R2 Score: ", np.round(knn_r2, 2))

Mean Absolute Error (MAE):  1.13
Mean Squared Error (MSE):  2.69
Root Mean Squared Error (RMSE):  1.64
R2 Score:  -0.33


In [40]:
ada_reg = AdaBoostRegressor(n_estimators = 100, learning_rate=0.5)

ada_reg.fit(train_df, train_y)

ada_reg_pred = ada_reg.predict(pd.DataFrame(test_df))
ada_reg_mae = mean_absolute_error(test_y, np.round(ada_reg_pred, 1))
ada_reg_mse = mean_squared_error(test_y, np.round(ada_reg_pred, 1))
ada_reg_mape = mean_absolute_percentage_error(test_y, np.round(ada_reg_pred,1), multioutput = "uniform_average")
ada_reg_rmse = mean_squared_error(test_y, np.round(ada_reg_pred, 1), squared=False)
ada_reg_r2 = r2_score(test_y, ada_reg_pred)

print("Mean Absolute Error (MAE): ", np.round(ada_reg_mae, 2))
print("Mean Squared Error (MSE): ", np.round(ada_reg_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(ada_reg_rmse, 2))
print("Mean Absolute Percentage Error (MAPE): ", np.round(ada_reg_mape, 2))
print("R2 Score: ", np.round(ada_reg_r2, 2))

Mean Absolute Error (MAE):  1.37
Mean Squared Error (MSE):  2.37
Root Mean Squared Error (RMSE):  1.54
Mean Absolute Percentage Error (MAPE):  509182488482297.2
R2 Score:  -0.2


In [41]:
gb_reg = GradientBoostingRegressor(n_estimators = 50, max_depth=5)

gb_reg.fit(train_df, train_y)

gb_reg_pred = gb_reg.predict(pd.DataFrame(test_df))
gb_reg_mae = mean_absolute_error(test_y, np.round(gb_reg_pred, 1))
gb_reg_mse = mean_squared_error(test_y, np.round(gb_reg_pred, 1))
gb_reg_mape = mean_absolute_percentage_error(test_y, np.round(gb_reg_pred,1), multioutput = "uniform_average")
gb_reg_rmse = mean_squared_error(test_y, np.round(gb_reg_pred, 1), squared=False)
gb_reg_r2 = r2_score(test_y, gb_reg_pred)

print("Mean Absolute Error (MAE): ", np.round(gb_reg_mae, 2))
print("Mean Squared Error (MSE): ", np.round(gb_reg_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(gb_reg_rmse, 2))
print("Mean Absolute Percentage Error (MAPE): ", np.round(gb_reg_mape, 2))
print("R2 Score: ", np.round(gb_reg_r2, 2))

Mean Absolute Error (MAE):  1.08
Mean Squared Error (MSE):  1.83
Root Mean Squared Error (RMSE):  1.35
Mean Absolute Percentage Error (MAPE):  556975790650310.6
R2 Score:  0.08


In [42]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', seed = 123)

xg_reg.fit(train_df, train_y)

# print("Best parameters: ", xg_boost_grid_search.best_params_)
# print("Best score: ", xg_boost_grid_search.best_score_)
xg_reg_pred = xg_reg.predict(pd.DataFrame(test_df))
xg_reg_mae = mean_absolute_error(test_y, np.round(xg_reg_pred, 1))
xg_reg_mse = mean_squared_error(test_y, np.round(xg_reg_pred, 1))
xg_reg_mape = mean_absolute_percentage_error(test_y, np.round(xg_reg_pred,1), multioutput = "uniform_average")
xg_reg_rmse = mean_squared_error(test_y, np.round(xg_reg_pred, 1), squared=False)
xg_reg_r2 = r2_score(test_y, xg_reg_pred)

print("Mean Absolute Error (MAE): ", np.round(xg_reg_mae, 2))
print("Mean Squared Error (MSE): ", np.round(xg_reg_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(xg_reg_rmse, 2))
print("Mean Absolute Percentage Error (MAPE): ", np.round(xg_reg_mape, 2))
print("R2 Score: ", np.round(xg_reg_r2, 2))

Mean Absolute Error (MAE):  1.16
Mean Squared Error (MSE):  2.2
Root Mean Squared Error (RMSE):  1.48
Mean Absolute Percentage Error (MAPE):  514697100270914.06
R2 Score:  -0.11
